In [1]:
import pandas as pd
from yelpapi import YelpAPI

In [30]:
df = pd.read_csv("C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/data/stockholm_housing_df_RAW.csv")

# 1. Process columns


In [241]:
df = pd.read_csv("C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/data/stockholm_housing_df_RAW.csv")

df['fee'] = df['fee'].str.rstrip(' kr/mån')

#Drop 'location' column, as it is redundant
df = df.drop(columns=["location"])

#Split size of apartment and number of rooms into separate columns
df[['size', 'rooms']] = df['size:rooms'].str.split("  ", 1, expand=True)
df = df.drop(columns=["size:rooms"])
#Clean up the columns size and room, removing m2 and "rum"
df['size'] = df['size'].str.rstrip('m²')
df["rooms"] = df["rooms"].str.rstrip(' rum')

#Split adress and floor of apt into separate columns
df[['adress', 'floor']] = df['adress'].str.split(",", 1, expand=True)

#Clean up sold_date, sale_price, value_dev, ppsqm
df["sold_date"] = df["sold_date"].str.lstrip('såld')
df["sale_price"] = df["sale_price"].str.lstrip('slutpris ')
df["sale_price"] = df["sale_price"].str.rstrip(' kr')
df["ppsqm"] = df["ppsqm"].str.rstrip(" kr/m²")
df["value_dev"] = df["value_dev"].str.rstrip(" %")
df["value_dev"] = df["value_dev"].str.lstrip("+")

# Introduce dummy variables in the following way: Balkong = 1, Hiss = 2, Balkong&Hiss = 3, Uteplats = 4
df["features"] = df.features.map( {'balkong':1 , 'hiss':2, 'balkong&hiss':3, 'uteplats': 4, 'NaN':5} )

#Remove everything except numerical values from floor
df['floor'] = df['floor'].str.extract('(\d+)', expand=False)

#Bug where value_dev ends up in wrong columns, due to old posting
bad_index = df[df["ppsqm"].str.find("%") != -1].index
df = df.drop(not_index, axis=0)

column_list = ["ppsqm", "size", "rooms", "sale_price","fee"]

def fix_numeric(column_list, df):
    new_df = df.copy()
    for i in column_list: 
        new_df[i] = new_df[i].str.replace(" ", "")
        new_df[i] = pd.to_numeric(new_df[i] , errors='coerce')
        new_df = new_df.dropna(subset=[i])
        new_df[i] = new_df[i].astype('int')
    return new_df

cleaned_housing_df = fix_numeric(column_list,df.copy() )
#Reset index after removing rows
cleaned_housing_df = cleaned_housing_df.reset_index(drop=True)
cleaned_housing_df.to_csv("stockholm_housing_df_CLEANED.csv",index=False)


In [242]:
cleaned_housing_df

,adress,fee,features,sale_price,sold_date,value_dev,ppsqm,district,size,rooms,floor
0,frejgatan 50,2391,1.0,6450000,15 januari 2022,NaN,153571,vasastan,42,2,None
1,rådmansgatan 86,2021,4.0,7200000,15 januari 2022,13,122034,vasastan,59,2,None
2,sankt eriksgatan 109,436,2.0,3150000,14 januari 2022,19,126000,vasastan,25,1,None
3,torsgatan 58,1733,2.0,3435000,14 januari 2022,15,110806,vasastan,31,1,1
4,torsplan 8,3950,3.0,10000000,14 januari 2022,11,153846,vasastan,65,3,9
...,...,...,...,...,...,...,...,...,...,...,...
10471,kammakargatan 70,1169,NaN,2800000,19 mars 2013,13,75676,norrmalm,37,2,3
10472,olofsgatan 18,917,NaN,2360000,28 februari 2013,8,73750,norrmalm,32,1,3
10473,regeringsgatan 70 d,3231,NaN,3600000,17 februari 2013,3,52174,norrmalm,69,3,None
10474,drottninggatan 80,2647,NaN,3250000,22 januari 2013,5,73864,norrmalm,44,2,3


In [237]:
#size, rooms, sale_price
column_list = ["ppsqm", "size", "rooms", "sale_price"]
new_df = fix_numeric(column_list, df)


C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [238]:
new_df

,adress,fee,features,sale_price,sold_date,value_dev,ppsqm,district,size,rooms,floor
0,frejgatan 50,2391,1.0,6450000,15 januari 2022,NaN,153571,vasastan,42,2,None
2,rådmansgatan 86,2021,4.0,7200000,15 januari 2022,13,122034,vasastan,59,2,None
3,sankt eriksgatan 109,436,2.0,3150000,14 januari 2022,19,126000,vasastan,25,1,None
5,torsgatan 58,1733,2.0,3435000,14 januari 2022,15,110806,vasastan,31,1,1
6,torsplan 8,3950,3.0,10000000,14 januari 2022,11,153846,vasastan,65,3,9
...,...,...,...,...,...,...,...,...,...,...,...
13438,kammakargatan 70,1169,NaN,2800000,19 mars 2013,13,75676,norrmalm,37,2,3
13440,olofsgatan 18,917,NaN,2360000,28 februari 2013,8,73750,norrmalm,32,1,3
13441,regeringsgatan 70 d,3231,NaN,3600000,17 februari 2013,3,52174,norrmalm,69,3,None
13442,drottninggatan 80,2647,NaN,3250000,22 januari 2013,5,73864,norrmalm,44,2,3


# Yelp API calls

In [196]:
yelp_api = YelpAPI("KulP_1xAbhj4PcwcltixYR5hz4qMJ2aarTp4uNP_bBED4CsgP1nqY0bZrDxRMMSsZYwqvirOQ1Dy--6v3Y2yS4lBVPmfebDVdXGukr74OZEKRNoivTBiORBJ0v_iYXYx")

#A 'suggested search area' of 500 is used, although, the docs tell us the following: 
#This field is used as a suggestion to the search. The actual search radius may be lower 
#than the suggested radius in dense urban areas, and higher in regions of less business density
#hopefully, it does give some quantifiable value as to the 'closeness' to points of interest a apartment has

def get_POI(adress):
    input_adress = adress
    response = yelp_api.search_query(location=input_adress, radius=500, limit=1)
    return response

In [200]:
yelp_dict = {}
def get_yelp_values():
    try:
        for i in range(0,1):
            response = get_POI(df["adress"][i])
            print("Getting values from adress:", df["adress"][i], "from row: " , i)
            
            lat = ( response['region']['center']['latitude'] )
            long = ( response['region']['center']['longitude'] )
            POI = ( response['total'])
            response_list = [lat,long,POI]
            
            yelp_dict[df["adress"][i]] = response_list
            

        return yelp_dict
    except:
        
        

    
get_yelp_values()
    
    
    
    

Getting values from adress: frejgatan 50 ---- row:  0


{'frejgatan 50': [59.34400390442908, 18.043670654296875, 82]}

In [198]:
yelp_dict

{}

In [164]:
yelp_api = YelpAPI("KulP_1xAbhj4PcwcltixYR5hz4qMJ2aarTp4uNP_bBED4CsgP1nqY0bZrDxRMMSsZYwqvirOQ1Dy--6v3Y2yS4lBVPmfebDVdXGukr74OZEKRNoivTBiORBJ0v_iYXYx")
response = yelp_api.search_query(location="torsgatan 25", radius=500, limit=1)

In [191]:
dict_test = {}
dict_test["liljegatan 8b"] = ["5", "6", "hej"]

dict_test

{'liljegatan 8b': ['5', '6', 'hej']}

In [192]:
df["adress"][10]

'vanadisvägen 4 k'